In [1]:
import numpy as np
import re


In [2]:
data = '''
우리나라 말이 중국과 달라서 문자로 서로 통하지 아니하니 이런 까닭으로 어리석은 백성들이 말하고자 하는 바가 있어도 마침내 제 뜻을 능히 펴지 못하는 사람이 많다.
내 이를 위해서 가엾이 생각하여 새로 스물여덟 자를 만드노니 사람마다 하여금 쉽게 익혀 매일 쓰기 편안하게 하고자 할 따름이다.
'''

In [3]:
def data_preprocessing(data):
  data = re.sub('[^가-힣]', ' ', data)
  tokens = data.split()
  vocab = list(set(tokens))
  vocab_size = len(vocab)

  word_to_ix = {word: i for i, word in enumerate(vocab)}
  ix_to_word = {i: word for i, word in enumerate(vocab)}

  return tokens, vocab_size, word_to_ix, ix_to_word

In [4]:
epochs = 10000
h_size = 100
seq_len = 3
learning_rate = 1e-2

In [5]:
tokens, vocab_size, word_to_ix, ix_to_word = data_preprocessing(data)

In [7]:
tokens[40]

'하고자'

In [ ]:
print("토큰 수:", len(tokens), "어휘 수:", vocab_size)

토큰 수: 43 어휘 수: 43


In [ ]:
word_to_ix

{'쉽게': 0,
 '못하는': 1,
 '자를': 2,
 '이를': 3,
 '통하지': 4,
 '하고자': 5,
 '펴지': 6,
 '아니하니': 7,
 '말이': 8,
 '새로': 9,
 '쓰기': 10,
 '문자로': 11,
 '사람이': 12,
 '바가': 13,
 '가엾이': 14,
 '이런': 15,
 '따름이다': 16,
 '생각하여': 17,
 '어리석은': 18,
 '익혀': 19,
 '까닭으로': 20,
 '뜻을': 21,
 '서로': 22,
 '능히': 23,
 '백성들이': 24,
 '달라서': 25,
 '우리나라': 26,
 '스물여덟': 27,
 '매일': 28,
 '편안하게': 29,
 '사람마다': 30,
 '말하고자': 31,
 '하는': 32,
 '하여금': 33,
 '마침내': 34,
 '있어도': 35,
 '많다': 36,
 '제': 37,
 '내': 38,
 '만드노니': 39,
 '위해서': 40,
 '중국과': 41,
 '할': 42}

In [ ]:
ix_to_word

{0: '쉽게',
 1: '못하는',
 2: '자를',
 3: '이를',
 4: '통하지',
 5: '하고자',
 6: '펴지',
 7: '아니하니',
 8: '말이',
 9: '새로',
 10: '쓰기',
 11: '문자로',
 12: '사람이',
 13: '바가',
 14: '가엾이',
 15: '이런',
 16: '따름이다',
 17: '생각하여',
 18: '어리석은',
 19: '익혀',
 20: '까닭으로',
 21: '뜻을',
 22: '서로',
 23: '능히',
 24: '백성들이',
 25: '달라서',
 26: '우리나라',
 27: '스물여덟',
 28: '매일',
 29: '편안하게',
 30: '사람마다',
 31: '말하고자',
 32: '하는',
 33: '하여금',
 34: '마침내',
 35: '있어도',
 36: '많다',
 37: '제',
 38: '내',
 39: '만드노니',
 40: '위해서',
 41: '중국과',
 42: '할'}

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    torch.backends.cudnn.benchmark = True

Using device: cuda


In [ ]:
# (입력 3단어 → 타깃 1단어) 샘플 생성
xs, ys = [], []
for i in range(len(tokens) - seq_len):
    seq = tokens[i:i+seq_len]
    target = tokens[i+seq_len]
    xs.append([word_to_ix[w] for w in seq])
    ys.append(word_to_ix[target])

x = torch.tensor(xs, dtype=torch.long)
y = torch.tensor(ys, dtype=torch.long)
dataset = TensorDataset(x, y)

In [ ]:
loader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    pin_memory=(device.type == "cuda")
)

In [ ]:
class RNN_basic(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers=2):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        # num_layers=2 → h1, h2
        self.rnn = nn.RNN(
            input_size=embed_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            nonlinearity='tanh',
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h0=None):
        # x: (B, T)
        emb = self.embed(x)               # (B, T, E)
        out, hn = self.rnn(emb, h0)       # out: (B, T, H)
        # 시퀀스의 마지막 타임스텝만 사용해 다음 단어 분포 예측
        last = out[:, -1, :]              # (B, H)
        logits = self.fc(last)            # (B, V)
        return logits, hn

In [ ]:
model = RNN_basic(vocab_size, embed_dim=h_size, hidden_size=h_size, num_layers=2).to(device)

In [ ]:
loss_func = torch.nn.CrossEntropyLoss()
optimzier = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
model.train()
for epoch in range(1, epochs + 1):
    total_loss = 0.0
    for xb, yb in loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        optimzier.zero_grad()
        logits, _ = model(xb)
        loss = loss_func(logits, yb)
        loss.backward()
        optimzier.step()

        total_loss += loss.item()

    if epoch % max(1, (epochs // 20)) == 0:  # 20등분 지점마다 로그
        avg = total_loss / len(loader)
        print(f"[{epoch:5d}/{epochs}] loss={avg:.4f}")

print("Training finished.")

[  500/10000] loss=0.0001
[ 1000/10000] loss=0.0000
[ 1500/10000] loss=0.0000
[ 2000/10000] loss=0.0000
[ 2500/10000] loss=0.0000
[ 3000/10000] loss=0.0000
[ 3500/10000] loss=0.0000
[ 4000/10000] loss=0.0000
[ 4500/10000] loss=0.0000
[ 5000/10000] loss=0.0000
[ 5500/10000] loss=0.0000
[ 6000/10000] loss=0.0000
[ 6500/10000] loss=0.0000
[ 7000/10000] loss=0.0000
[ 7500/10000] loss=0.0000
[ 8000/10000] loss=0.0000
[ 8500/10000] loss=0.0000
[ 9000/10000] loss=0.0000
[ 9500/10000] loss=0.0000
[10000/10000] loss=0.0000
Training finished.


## TEST

In [ ]:
def predict_next(word_triplet, topk=5):
    """3단어 시드로 다음 단어 상위 k개 예측"""
    model.eval()
    with torch.no_grad():
        idx = torch.tensor([[word_to_ix[w] for w in word_triplet]], dtype=torch.long).to(device)
        logits, _ = model(idx)
        probs = torch.softmax(logits, dim=-1).squeeze(0).cpu()
        top_probs, top_idx = probs.topk(topk)
        return [(ix_to_word[i.item()], float(p.item())) for i, p in zip(top_idx, top_probs)]

In [ ]:
def generate_text(seed_words, gen_len=10):
    """3단어 시드에서 시작해 gen_len개 단어를 연쇄 생성"""
    assert len(seed_words) == 3, "seed_words는 3단어여야 합니다."
    seq = seed_words[:]
    model.eval()
    with torch.no_grad():
        for _ in range(gen_len):
            idx = torch.tensor([[word_to_ix[w] for w in seq[-3:]]], dtype=torch.long).to(device)
            logits, _ = model(idx)
            next_id = torch.argmax(logits, dim=-1).item()
            next_word = ix_to_word[next_id]
            seq.append(next_word)
    return ' '.join(seq)

In [ ]:
# 데이터에 실제로 존재하는 3연속 단어를 시드로 써야 합니다.
# 예: tokens 안에서 보이는 인접 3단어를 하나 고릅니다.
seed = tokens[0:3]
print("시드:", seed)
print("다음 단어 Top-5:", predict_next(seed, topk=5))
print("연쇄 생성:", generate_text(seed, gen_len=15))

시드: ['우리나라', '말이', '중국과']
다음 단어 Top-5: [('달라서', 1.0), ('능히', 5.401183500453044e-09), ('만드노니', 3.862209219107626e-09), ('이런', 3.32748384401782e-09), ('백성들이', 3.31702820766111e-09)]
연쇄 생성: 우리나라 말이 중국과 달라서 문자로 서로 통하지 아니하니 이런 까닭으로 어리석은 백성들이 말하고자 하는 바가 있어도 마침내 제
